# **CH01.1. Word2Vec**

#### **`Paper Info`** : Efficient Estimation of Word Representations in Vector Space
#### $ \hspace{1.75cm} - $ Tomas Mikolov, Kai Chen, Greg Corrado, Jeffrey Dean (ICLR/2013)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|-------------|
| Research topic | 대규모 말뭉치 환경에서 효율적으로 단어 분산 표현을 학습하기 위한 log-linear 기반 CBOW·Skip-gram 모델 제안 |
| Key findings | $ \cdot{} $ (Word2Vec은) Non-linear hidden layer 제거 및 Hierarchical Softmax로 NNLM, RNNLM 대비 sample 별 학습 연산량 대폭 감소 <br> $ \cdot{} $ 문맥 확률 분포 학습 효과로 학습된 임베딩이 의미, syntactic 유사성을 반영 |
| Contributions | $ \cdot{} $ shallow log-linear 구조인 CBOW, skip-gram 모델 제시로 언어 모델 학습 효율 향상시킴 <br> $ \cdot{} $ Hierarchical Softmax로 출력 예측 복잡도 $O \big({} \log_{2}(V) \big){} $ 달성 <br> $ \cdot{} $ 대규모 데이터 실험으로 연속형 단어 표현의 실용성 입증함 |

<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ 기존 NNLM(Neural Network Language Model)은 예측 토큰 시점 $ \, t $, 고려하는 과거 단어의 개수(context window size)가 $ n $일 때 
##### $ \hspace{0.45cm} p(w_{t} | w_{t-n+1}, \cdots{}, w_{t-1}) $를 모델링하지만, 샘플 당 계산 복잡도가 높음
##### $ \hspace{0.15cm} \cdot{} $ RNNLM(RNN-Language Model)은 장기 의존성은 포착하지만 샘플 당 계산 복잡도가 높음
##### $ \hspace{0.15cm} \cdot{} $ 수십억 토큰 규모의 대규모 데이터셋에서 이러한 복잡도가 학습 병목으로 작용

#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ 연산 효율이 높은 확률적 언어 모델을 설계
##### $ \hspace{0.15cm} \cdot{} $ 단어 간 의미적 유사성과 문법적 규칙성을 반영한 고품질 임베딩 벡터를 학습
##### $ \hspace{0.15cm} \cdot{} $ 대규모 데이터셋에서도 학습이 가능한 확장성(scalability)을 달성

#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ 문맥 단어(context word)의 임베딩 평균으로 중심 단어(center word)를 예측해 맥락 기반의 단어 의미를 효율적으로 학습하는 CBOW 모델 제안
##### $ \hspace{0.15cm} \cdot{} $ 그 중심 단어를 기반으로 주변 단어를 예측해 희귀 단어의 표현력을 강화하고 의미적, 통사적 관계를 포착하는 Skip-gram 모델 제안
##### $ \hspace{0.15cm} \cdot{} $ Hierarchical Softmax 기법을 활용해 대규모 어휘에서의 학습 효율성과 표현 품질을 동시에 향상

<b></b>

> ## **방법론(Method)**

#### **(1) CBOW(Continuous Bag-of-Words)** :
##### $ \hspace{0.15cm} \cdot{} $ 주변 단어 집합 $ \, \mathcal{C}_{t} = \{ w_{t-c}, \cdots{}, w_{t-1}, w_{t+1}, \cdots{}, w_{t+c} \} $으로부터 중심 단어 $ w_{t} $를 예측할 확률을 최대화하는 것을 목표
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \theta^{*} = \arg\max_{\theta} \sum_{t=1}^{T} \log p(w_t | \mathcal{C}_{t}; \theta{}) $
##### $ \hspace{0.45cm} \text{where } \, \mathcal{C}_{t} : \text{input(context word) indices within window}, \;\; w_{t} : \text{output(center word) index} $
##### $ \hspace{0.45cm} \text{and } \, T : \text{token size}, \;\; c : \text{window size} $
##### $ \hspace{0.15cm} \cdot{} $ 주변 단어 임베딩의 평균을 통해 맥락 벡터 $ \, \bar{\textbf{v}}_{t} $를 구성해 조건부 확률 $ \, p(w_{t} | \bar{\textbf{v}}_{t}; \theta{}) $을 (Hierarchical) softmax로 계산
##### $ \hspace{0.15cm} \Rightarrow{} p(w_t | \mathcal{C}_{t}; \theta{}) \equiv{} p(w_{t} | \bar{\textbf{v}}_{t}) = \displaystyle{} \frac{\exp{}\big(\textbf{u}_{w_{t}}^{\top} \bar{\textbf{v}}_{t}\big)}{\displaystyle{} \sum_{w' \in{} \mathcal{V}} \exp{}\big(\textbf{u}_{w'}^{\top} \bar{\textbf{v}}_{t}\big)} $ 
##### $ \hspace{0.45cm} \text{where } \, \bar{\textbf{v}}_{t} =  \frac{1}{2c} \sum_{\substack{-c \le j \le c \\ j \neq 0}} \textbf{v}_{w_{t+j}} $
##### $ \hspace{0.45cm} \text{and } \, \textbf{v}_{w_{t+j}} : \text{input embedding vectors}, \;\; \textbf{u}_{w_{t}} : \text{output embedding vector} $
##### $ \hspace{0.45cm} \text{and } \, \textbf{v}_{w} : \text{final word embedding after training} $
##### $ \hspace{0.45cm} \text{and } \, \mathcal{V} : \text{vocabulary set}, \;\; |\mathcal{V}|=V $

##### **(`PLUS`) 계층적 소프트맥스(Hierarchical Softmax)** :
##### $ \hspace{0.15cm} \cdot{} $ 각 단어를 리프 노드(leaf node)로하는 이진 트리에서 내부 노드 $ \, n $ 마다 파라미터 벡터 $ \mathbf{u}_{n} $를 두고 경로를 따라 이진 분류 확률을 곱하는 방법
##### $ \hspace{0.15cm} \Rightarrow{} p(w | \mathbf{x} ) = \displaystyle{} \prod_{n \in{} \text{path}(w)} \sigma{} \big({} s(n,w) \mathbf{u}_{n}^{\top} \mathbf{x} \big){} $
##### $ \hspace{0.45cm} \text{where } \, \text{path}(w) : \text{root}\to\text{leaf}(w), \;\; s(n,w) \in{} \{ -1,+1 \}, \;\; \mathbf{u}_{n} \in{} \mathbb{R}^{D}, \;\; \sigma{}(x) = \frac{1}{1+\exp{}(-x)} $
##### $ \hspace{0.15cm} \cdot{} $ 허프만 코드(Huffman code)로 빈도 높은 단어의 경로를 짧게 하여 기대 연산량을 추가 절감함

#### **(2) Skip-gram** :
##### $ \hspace{0.15cm} \cdot{} $ 중심 단어 $ w_{t} $로부터 주변 단어 집합 $ \, \mathcal{C}_{t} = \{ w_{t-c}, \cdots{}, w_{t-1}, w_{t+1}, \cdots{}, w_{t+c} \} $를 예측을 목표
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \theta^{*} = \arg\max_{\theta{}} \sum_{t=1}^{T} \sum_{\substack{-c \le j \le c \\ j \neq 0}} \log p(w_{t+j} \mid w_{t}; \theta{}) $
##### $ \hspace{0.15cm} \cdot{} $ 중심 단어 임베딩 $ \, \textbf{v}_{w_t} $를 통해 주변 단어의 조건부 확률 $ \, p(w_{t+j} | w_t; \theta{}) $을 (Hierarchical) softmax로 계산
##### $ \hspace{0.15cm} \Rightarrow{} p(w_{t+j} | w_t; \theta{}) = \displaystyle{} \frac{\exp{}\big(\textbf{u}_{w_{t+j}}^{\top} \textbf{v}_{w_t}\big)}{\displaystyle{} \sum_{w' \in{} \mathcal{V}} \exp{}\big(\textbf{u}_{w'}^{\top} \textbf{v}_{w_t}\big)} $

<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | Google News $ + $ additional corpus |
| Vocabulary size($ V $)| $ \sim{}1\text{M} \, $ (most frequent words) |
| token size($ T $) | $ \sim{} 1.6\text{B} \, $ and $ 6\text{B} $ |
| Evaluation dataset | Semantic–Syntactic Word Relationship(Google analogy) |
##### $ \hspace{0.15cm} \cdot{} $ 모델
| Item | Description |
|------|--------------|
| Model type | Continuous Bag-of-Words (CBOW) / Skip-gram |
| Architecture | $ \cdot{} $ Input: one-hot <br> $ \cdot{} $ Projection: avg (CBOW) or single (Skip-gram), D-dim <br> $ \cdot{} $ Output: Hierarchical Softmax |
| Hyper parameter | $ \cdot{} $ Embedding dimension($ D $) : $ 50 / 100 / 300 / 1000 $ <br> $ \cdot{} $ Context window size($ c $) : CBOW $ c = 4 $, Skip-gram max $ c=10 $ (distance-weighted) <br> $ \cdot{} $ Optimizer : SGD <br> $ \cdot{} $ Learning rate : $ 0.025 $ (linearly decayed) |
| Parameter size | $ \cdot{} $ Parameter size : $ \approx{} 2VD $ |
##### $ \hspace{0.15cm} \cdot{} $ 훈련 및 평가
| Item | Description |
|------|--------------|
| Loss function | Negative log-likelihood |
| Evaluation metric | Analogy exact-match accuracy |
| Baseline models | Feed-forward NNLM, RNNLM |
| Training hardware | Multi-threaded CPU (single-machine) & Distributed (DistBelief) |
| Training time | ? | 

#### **(2) 실험 결과** :
##### $ \hspace{0.15cm} \cdot{} $ 임베딩 차원 $ D $와 학습 데이터 크기(토큰 수 $ T $)를 함께 증가시킬 때, 단독으로 어느 한쪽만 늘리는 경우보다 품질 향상이 가장 큼
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram 모델은 semantic 관계에서, CBOW는 syntactic 관계에서 상대적 강점을 가짐
##### $ \hspace{0.15cm} \cdot{} $ NNLM 및 RNNLM 대비 은닉층이 없는 구조로 동일한 시간 내 더 많은 토큰을 처리하며, hierarchical softmax를 통해 학습 효율을 극대화함

##### **(`PLUS`) 계산 복잡도(Computational Complexity; $ Q $) 비교** :
##### $ \hspace{0.15cm} \cdot{} \, Q_{\text{NNLM}} = N D + N D H + H V  $
##### $ \hspace{0.15cm} \cdot{} \, Q_{\text{RNNLM}} = H^{2} + H V  $
##### $ \hspace{0.15cm} \cdot{} \, Q_{\text{CBOW}} = N D + D \log_{2}{V} $
##### $ \hspace{0.15cm} \cdot{} \, Q_{\text{SG}} = C ( D + D \log_{2}{V} ) $
##### $ \hspace{0.3cm} \text{where } \, N = 2c : \text{context word size}, \;\; D : \text{embedding dim}, \;\; H : \text{hidden layer dim} $
##### $ \hspace{0.3cm} \text{and } \, V : \text{vocabulary size}, \;\; C : \text{max window radius} $

<b></b>

> ## **결론(Conclusion)**

#### **(1)** CBOW는 평균 맥락 표현으로 계산을 단순화하여 효율을 확보

#### **(2)** Skip-gram은 희소하지만 정보량 큰 주변 단서까지 개별 예측해 semantic 관계에 유리, CBOW는 평균 맥락으로 syntactic 관계에 유리함

#### **(3)** Hierarchical Softmax 도입으로 대규모 Vocabulary size로 인한 계산 병목을 완화

<b></b>

> ## **부록(Appendix)**

<b></b>

> ## **비평(Commentary)**

<b></b>

> ## **참고 문헌(Reference)**

#### **(1) 관련 후속 논문** :
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**

#### **(2) 기타 참고 자료** :
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**